# Part 1: Sentiment Analysis

In [ ]:
import pandas as pd

# Load the CSV file
file_path = './data/NEWS_YAHOO_stock_prediction.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe
data.head()

In [ ]:
# Step 1: Remove unnecessary column
data.drop(columns=['Unnamed: 0'], inplace=True)

# Step 2: Remove duplicate texts
data.drop_duplicates(subset=['title', 'content'], inplace=True)

# Step 3: Remove rows with large amount of spaces or empty texts in 'title' and 'content'
data = data[~data['title'].str.isspace()]
data = data[~data['content'].str.isspace()]
data.dropna(subset=['title', 'content'], inplace=True)

# Check the dataframe after these preprocessing steps
data.info()

# Step 5: Check for invalid numeric data
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
data[numeric_columns].describe()


In [ ]:
# (optional) set proxy
import subprocess
import os

result = subprocess.run('bash -c "source ~/clash_dir/set && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
output
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

# Load the FinBERT model and tokenizer
checkpoint = 'yiyanghkust/finbert-tone'
tokenizer = BertTokenizer.from_pretrained(checkpoint)
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

# Create a pipeline for sentiment analysis
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, max_length=512, truncation=True, device=0)

In [ ]:
# Function to apply sentiment analysis to a dataframe
def apply_sentiment_analysis(df, nlp, text_column='content'):
    """
    Apply sentiment analysis to a column in a dataframe.
    
    Args:
    df (pd.DataFrame): Dataframe containing the text data.
    nlp (pipeline): HuggingFace pipeline for sentiment analysis.
    text_column (str): Name of the column containing text data.

    Returns:
    pd.DataFrame: Dataframe with a new column 'sentiment' containing the analysis results.
    """
    # Apply sentiment analysis to each row in the text column
    sentiments = []
    for text in df[text_column]:
        try:
            result = nlp(text)
            sentiments.append(result[0]['label'])
        except Exception as e:
            print(f"Error in processing text: {e}")
            sentiments.append('Error')

    # Add the sentiments as a new column in the dataframe
    df['sentiment'] = sentiments
    return df


In [ ]:
from tqdm.auto import tqdm
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def apply_sentiment_analysis_parallel(df, nlp, text_column='content', batch_size=10):
    """
    Apply sentiment analysis in parallel to a column in a dataframe.

    Args:
    df (pd.DataFrame): Dataframe containing the text data.
    nlp (pipeline): HuggingFace pipeline for sentiment analysis.
    text_column (str): Name of the column containing text data.
    batch_size (int): Number of texts to process in parallel.

    Returns:
    pd.DataFrame: Dataframe with a new column 'sentiment' containing the analysis results.
    """
    # Define a function to process a batch of texts
    def process_batch(texts):
        return [nlp(text)[0]['label'] for text in texts]

    # Break the texts into batches
    batches = [df[text_column][i:i + batch_size] for i in range(0, len(df), batch_size)]

    # Process batches in parallel
    sentiments = []
    with ThreadPoolExecutor() as executor:
        for batch_result in tqdm(executor.map(process_batch, batches), total=len(batches)):
            sentiments.extend(batch_result)

    # Add the sentiments as a new column in the dataframe
    df['sentiment'] = sentiments
    return df


In [ ]:
# Example usage of the function
# Note: You will run this on your local machine as it requires GPU support
sample_texts = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]
sample_df = pd.DataFrame(sample_texts, columns=['content'])
apply_sentiment_analysis(sample_df, nlp)

In [ ]:
# Assuming the apply_sentiment_analysis function is defined as shown previously

# Step 1: Apply sentiment analysis to the dataset
# This step should be done on your local machine due to the requirement of GPU support
data = apply_sentiment_analysis_parallel(data, nlp)

# Step 2: Prepare data for the prediction model
# Here we'll assume the sentiment analysis has been applied and 'sentiment' column is added to the data

# We might want to convert sentiments to numerical values for model training
sentiment_mapping = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
data['sentiment_numeric'] = data['sentiment'].map(sentiment_mapping)

In [ ]:
# Example code to save the processed DataFrame to a CSV file
data.to_csv('./data/dataset_with_sentiment.csv', index=False)


In [ ]:
# Adjust display settings for better visualization of samples
pd.set_option('display.max_colwidth', 200)  # Adjust the width to fit longer texts

# Display some random samples with formatted output
sample_data = data.sample(n=10)[['content', 'sentiment']]

# Print each sample in a more readable format
for index, row in sample_data.iterrows():
    print(f"Sample {index}:")
    print(f"Content: {row['content']}")
    print(f"Sentiment: {row['sentiment']}\n")


In [ ]:
# Assuming 'data' is your DataFrame with 'sentiment' and 'label' columns
# Calculate the proportion of each sentiment category
sentiment_counts = data['sentiment'].value_counts(normalize=True) * 100

# Calculate the proportion of each label
label_counts = data['label'].value_counts(normalize=True) * 100

# Print the results
print("Sentiment Distribution (%):")
print(sentiment_counts)
print("\nLabel Distribution (%):")
print(label_counts)

# For additional insights, we can also look at the cross-tabulation of sentiment and label
crosstab = pd.crosstab(data['sentiment'], data['label'], normalize='index') * 100
print("\nCross-Tabulation of Sentiment and Label (%):")
print(crosstab)

In [4]:
# read for existed csv
import pandas as pd
data = pd.read_csv('./data/dataset_with_sentiment.csv')

# Convert the 'Date' column to datetime format and sort the dataframe by 'Date'
data['Date'] = pd.to_datetime(data['Date'])
data_sorted = data.sort_values(by='Date')

In [5]:
# 按 'Date' 和 'sentiment' 分组，然后计算每个类别的 category 为news和opinion的数量
category_news_per_day_sentiment = data_sorted[data_sorted['category'] == 'news'].groupby(['Date', 'sentiment']).size().unstack().fillna(0)
category_opinion_per_day_sentiment = data_sorted[data_sorted['category'] == 'opinion'].groupby(['Date', 'sentiment']).size().unstack().fillna(0)
# 分别计算news和opinion的total
category_news_total_per_day_sentiment = data_sorted[data_sorted['category'] == 'news'].groupby(['Date']).size()
category_opinion_total_per_day_sentiment = data_sorted[data_sorted['category'] == 'opinion'].groupby(['Date']).size()

data_sorted = data_sorted.set_index('Date')
data_sorted['P_news_pos'] = category_news_per_day_sentiment['Positive'].reindex(data_sorted.index) / category_news_total_per_day_sentiment.reindex(data_sorted.index)
data_sorted['P_news_neg'] = category_news_per_day_sentiment['Negative'].reindex(data_sorted.index) / category_news_total_per_day_sentiment.reindex(data_sorted.index)
data_sorted['P_op_pos'] = category_opinion_per_day_sentiment['Positive'].reindex(data_sorted.index) / category_opinion_total_per_day_sentiment.reindex(data_sorted.index)
data_sorted['P_op_neg'] = category_opinion_per_day_sentiment['Negative'].reindex(data_sorted.index) / category_opinion_total_per_day_sentiment.reindex(data_sorted.index)
data_sorted = data_sorted.reset_index()

In [6]:
daily_data = data_sorted.groupby('Date').last()

# Shift the 'Open' column to get the next day's opening price
daily_data['Next_Open'] = daily_data['Open'].shift(-1)

# Drop the last row as it will not have a 'Next_Open' value
daily_data = daily_data[:-1]

In [7]:
date_to_query = pd.to_datetime('2016-10-28')
daily_data.loc[(date_to_query)]

ticker                                                            AAPL
category                                                       opinion
title                   Dollar Sidelined  Krona Stabilizes  Rates Firm
content              The main development here in the last full wee...
Open                                                         28.467501
High                                                           28.8025
Low                                                          28.362499
Close                                                            28.43
Adj Close                                                    26.514231
Volume                                                       151446800
label                                                                0
sentiment                                                      Neutral
sentiment_numeric                                                    0
P_news_pos                                                         0.0
P_news

In [8]:
date_to_query = pd.to_datetime('2016-10-28')
data_sorted.loc[data_sorted['Date'] == date_to_query]

,Date,ticker,category,title,content,Open,High,Low,Close,Adj Close,Volume,label,sentiment,sentiment_numeric,P_news_pos,P_news_neg,P_op_pos,P_op_neg
3891,2016-10-28,AAPL,opinion,Tech Earnings Roundup Amazon Alphabet Apple...,On this special short earnings edition of the...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Positive,1,0.0,0.0,0.5,0.125
3892,2016-10-28,AAPL,opinion,Videogame Stock Roundup Earnings In Focus Ac...,Earnings reports were in the limelight this we...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Negative,-1,0.0,0.0,0.5,0.125
3893,2016-10-28,AAPL,opinion,Netflix And Amazon Video Absent From Apple s N...,During Apple Inc s NASDAQ AAPL recent Mac...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Neutral,0,0.0,0.0,0.5,0.125
3894,2016-10-28,AAPL,opinion,Top Ranked ETFs To Buy On Alphabet s Robust Q3...,Google s parent company Alphabet Inc s NASD...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Positive,1,0.0,0.0,0.5,0.125
3895,2016-10-28,AAPL,opinion,The Zacks Stocks In The News Blog Highlights ...,For Immediate Release\nChicago IL October 28 ...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Positive,1,0.0,0.0,0.5,0.125
3896,2016-10-28,AAPL,news,Amazon targets Chinese demand for overseas sho...,BEIJING Reuters Amazon com Inc NASDAQ AMZ...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Neutral,0,0.0,0.0,0.5,0.125
3897,2016-10-28,AAPL,opinion,An End To The Earnings Recession,The growth picture emerging from the Q3 earnin...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Positive,1,0.0,0.0,0.5,0.125
3898,2016-10-28,AAPL,opinion,Is Apple s Stock Cheap Not To My Eyes,Apple NASDAQ AAPL reported earnings this wee...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Neutral,0,0.0,0.0,0.5,0.125
3899,2016-10-28,AAPL,opinion,Dollar Sidelined Krona Stabilizes Rates Firm,The main development here in the last full wee...,28.467501,28.8025,28.362499,28.43,26.514231,151446800,0,Neutral,0,0.0,0.0,0.5,0.125


In [9]:
date_to_query = pd.to_datetime('2020-01-23')
daily_data.loc[(date_to_query)]

ticker                                                            AAPL
category                                                          news
title                Apple Boosts Chip Orders From Main Foundry Sup...
content              Apple s  NASDAQ AAPL  iPhone 11 has been selli...
Open                                                         79.480003
High                                                         79.889999
Low                                                          78.912498
Close                                                        79.807503
Adj Close                                                    78.315315
Volume                                                       104472000
label                                                                1
sentiment                                                     Positive
sentiment_numeric                                                    1
P_news_pos                                                    0.461538
P_news

In [10]:
date_to_query = pd.to_datetime('2018-05-06')
category_to_query = 'news'
data_sorted.loc[(data_sorted['Date'] == date_to_query) & (data_sorted['category'] == category_to_query)]

,Date,ticker,category,title,content,Open,High,Low,Close,Adj Close,Volume,label,sentiment,sentiment_numeric,P_news_pos,P_news_neg,P_op_pos,P_op_neg
9679,2018-05-06,AAPL,news,Apple and Buffett saw value and acted,By Noel Randewich SAN FRANCISCO Reuters Ap...,48.267502,48.485001,48.09,48.327499,46.396736,86264000,1,Neutral,0,0.0,0.0,0.814815,0.12963
9680,2018-05-06,AAPL,news,Buffett craves more Apple shares endorses its...,By Trevor Hunnicutt and Jonathan Stempel OMAHA...,48.267502,48.485001,48.09,48.327499,46.396736,86264000,1,Neutral,0,0.0,0.0,0.814815,0.12963


In [11]:
daily_data['P_news_neg'].fillna(0, inplace=True)
daily_data['P_news_pos'].fillna(0, inplace=True)
daily_data['P_op_neg'].fillna(0, inplace=True)
daily_data['P_op_pos'].fillna(0, inplace=True)

In [18]:
# 不直接删除，而是存到新的df中。保留'date'列
columns_to_keep = [col for col in daily_data.columns if col not in ['ticker', 'Adj Close', 'sentiment', 'sentiment_numeric', 'title', 'category', 'content', 'label']]
daily_data_selected = daily_data[columns_to_keep]
# 将'Date'列设置为daily_data_selected的索引，并将其排到第一位
daily_data_selected = daily_data_selected.reset_index()
daily_data_selected.set_index('Date', inplace=True)
daily_data_selected = daily_data_selected.sort_index()

In [19]:
daily_data_selected.head()

,Open,High,Low,Close,Volume,P_news_pos,P_news_neg,P_op_pos,P_op_neg,Next_Open
Date,,,,,,,,,,
2012-07-23,21.228571,21.639286,20.989643,21.565357,487975600,0.0,0.0,0.333333,0.333333,21.692142
2012-07-24,21.692142,21.774286,21.375357,21.461430,565132400,0.0,0.0,0.000000,0.000000,20.536072
2012-07-27,20.536072,20.922501,20.413929,20.898571,403936400,0.0,0.0,0.000000,1.000000,21.104286
2012-07-30,21.104286,21.408571,20.993570,21.251072,379142400,0.0,0.0,0.000000,1.000000,21.543928
2012-07-31,21.543928,21.846430,21.525715,21.812857,462327600,0.0,0.0,0.500000,0.000000,23.102858


In [20]:
daily_data_selected.to_csv('./data/dataset_for_model.csv', index=True)

# Part 2: Stock price prediction

In [16]:
import pandas as pd
daily_data_selected = pd.read_csv('./data/dataset_for_model.csv')

In [3]:
daily_data_selected

,Open,High,Low,Close,Volume,P_news_pos,P_news_neg,P_op_pos,P_op_neg,Next_Open
0,21.228571,21.639286,20.989643,21.565357,487975600,0.000000,0.000000,0.333333,0.333333,21.692142
1,21.692142,21.774286,21.375357,21.461430,565132400,0.000000,0.000000,0.000000,0.000000,20.536072
2,20.536072,20.922501,20.413929,20.898571,403936400,0.000000,0.000000,0.000000,1.000000,21.104286
3,21.104286,21.408571,20.993570,21.251072,379142400,0.000000,0.000000,0.000000,1.000000,21.543928
4,21.543928,21.846430,21.525715,21.812857,462327600,0.000000,0.000000,0.500000,0.000000,23.102858
...,...,...,...,...,...,...,...,...,...,...
1648,79.067497,79.684998,78.750000,79.682503,137816400,0.500000,0.250000,0.700000,0.000000,79.297501
1649,79.297501,79.754997,79.000000,79.142502,110843200,0.200000,0.200000,0.523810,0.238095,79.644997
1650,79.644997,79.997498,79.327499,79.425003,101832400,0.333333,0.200000,0.666667,0.066667,79.480003
1651,79.480003,79.889999,78.912498,79.807503,104472000,0.461538,0.076923,0.909091,0.000000,80.062500


In [2]:
# 选择特征和目标
# features = daily_data_selected.drop('Next_Open', axis=1)
features = daily_data_selected.drop(['Next_Open', 'P_news_pos', 'P_news_neg', 'P_op_pos', 'P_op_neg'], axis=1)
# Open作为预测目标
target = daily_data_selected['Open']
features.tail()

,Open,High,Low,Close,Volume
1648,79.067497,79.684998,78.750000,79.682503,137816400
1649,79.297501,79.754997,79.000000,79.142502,110843200
1650,79.644997,79.997498,79.327499,79.425003,101832400
1651,79.480003,79.889999,78.912498,79.807503,104472000
1652,80.062500,80.832497,79.379997,79.577499,146537600


In [ ]:
target.tail()

In [ ]:
# normalization
from sklearn.preprocessing import MinMaxScaler

# Apply the MinMaxScaler to the features and target
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

# fit_transform根据数据计算缩放参数
scaled_features = scaler_features.fit_transform(features)
scaled_target = scaler_target.fit_transform(target.values.reshape(-1, 1))

# 保存缩放参数
import joblib
joblib.dump(scaler_features, './model/scaler_features.pkl')
joblib.dump(scaler_target, './model/scaler_target.pkl')

# Create new DataFrames with the scaled features and target
scaled_features_df = pd.DataFrame(scaled_features, columns=features.columns)
scaled_target_df = pd.DataFrame(scaled_target, columns=['Open'])

In [ ]:
scaled_target_df

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
def create_sequences(features, targets, seq_length):
    """
    Create sequences of specified length from time series data.

    Args:
    features (np.array): The feature data.
    targets (np.array): The target data.
    seq_length (int): The length of the sequence.

    Returns:
    np.array: Sequences of features.
    np.array: Corresponding targets for each sequence.
    """
    xs, ys = [], []
    for i in range(len(features) - seq_length):
        x = features[i:(i + seq_length)]
        y = targets[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Example sequence length
seq_length = 20

# Create sequences
features_seq, target_seq = create_sequences(scaled_features, scaled_target, seq_length)

# Convert sequences to Tensor
features_tensor = torch.tensor(features_seq, dtype=torch.float32)
target_tensor = torch.tensor(target_seq, dtype=torch.float32)

In [ ]:
# 划分训练集、验证集、测试集
train_features, test_features, train_target, test_target = train_test_split(
    scaled_features, scaled_target, test_size=0.2, random_state=42
)

val_features, test_features, val_target, test_target = train_test_split(
    test_features, test_target, test_size=0.5, random_state=42
)

# 创建TensorDataset
train_dataset = TensorDataset(torch.tensor(train_features, dtype=torch.float32),
                              torch.tensor(train_target, dtype=torch.float32))

val_dataset = TensorDataset(torch.tensor(val_features, dtype=torch.float32),
                            torch.tensor(val_target, dtype=torch.float32))

test_dataset = TensorDataset(torch.tensor(test_features, dtype=torch.float32),
                             torch.tensor(test_target, dtype=torch.float32))

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# 使用普通的LSTM模型，不使用注意力机制
class SimpleLSTM(nn.Module):
    def __init__(self, input_dim, hidden_size, num_layers, output_dim, dropout=0.2):
        super(SimpleLSTM, self).__init__()
        self.hidden_size = hidden_size

        # LSTM层
        self.lstm = nn.LSTM(input_dim, hidden_size, num_layers, 
                            batch_first=True, dropout=dropout if num_layers > 1 else 0)
        
        # 全连接层
        self.fc = nn.Linear(hidden_size, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # 取最后一个时间步的输出
        output = self.fc(lstm_out[:, -1, :])
        return output

In [ ]:
# 超参数
input_dim = scaled_features_df.shape[1]  # 特征数量
hidden_size = 100  # 隐藏状态中的特征数量，可以调整
num_layers = 4    # 堆叠的LSTM层的数量
output_dim = 1    # 输出维度的数量（预测一个值）

# 实例化模型时添加Dropout
model = SimpleLSTM(input_dim, hidden_size, num_layers, output_dim, dropout=0.2)

In [ ]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs
num_epochs = 60  # Can be adjusted

# 在训练循环中添加对验证集的检查
best_val_loss = float('inf')
# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    # 在每个epoch结束后评估验证集
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_outputs = model(val_inputs)
            val_loss += criterion(val_outputs, val_labels).item()
    val_loss /= len(val_loader)
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
    print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss}')
    
    # 检查是否有更低的验证损失
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), './model/best_model.pth')  # 保存最好的模型

In [ ]:
val_dataset[:][0]

In [ ]:
val_dataset[:][1]

In [ ]:
# 加载最佳模型
model.load_state_dict(torch.load('./model/best_model.pth'))

# 从测试集中提取特征和目标
# 改为val_dataset
test_features = val_dataset[:][0]
test_target = val_dataset[:][1]

# 创建滑动窗口
test_features_seq, test_target_seq = create_sequences(test_features.numpy(), test_target.numpy(), seq_length)

# 将序列转换为Tensor
test_features_tensor = torch.tensor(test_features_seq, dtype=torch.float32)
test_target_tensor = torch.tensor(test_target_seq, dtype=torch.float32)

# 使用模型进行预测
model.eval()
with torch.no_grad():
    test_predictions = model(test_features_tensor).numpy()

# 反缩放预测值
test_predictions = scaler_target.inverse_transform(test_predictions).flatten()

# 反缩放真实目标值
test_target = scaler_target.inverse_transform(test_target_tensor.numpy()).flatten()

# 绘制实际股价和预测股价的对比图
plt.figure(figsize=(10, 6))
plt.plot(test_target[seq_length:], label='Actual Prices', color='blue')
plt.plot(test_predictions, label='Predicted Prices', color='red')
plt.title('Predicted vs Actual Prices')
plt.xlabel('Time Step')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(test_target, test_predictions))
print('Test RMSE: ', rmse)

In [ ]:
import matplotlib.dates as mdates

# 加载最佳模型
model.load_state_dict(torch.load('./model/best_model.pth'))

# 提取最后n个数据点
test_length = 100
last_n = daily_data_selected[-test_length:]

# 保存最后n个日期，这将用于绘图
last_n_dates = last_n.index

# 删除日期和目标列以准备缩放
last_n_features = last_n.drop(['Next_Open'], axis=1)
last_n_target = last_n['Open']

# 仅在scaler_features/scaler_target在当前内存中不存在时，才从文件中加载保存的缩放器状态
if 'scaler_features' not in locals():
    scaler_features = joblib.load('./model/scaler_features.pkl')
if 'scaler_target' not in locals():
    scaler_target = joblib.load('./model/scaler_target.pkl')

# 缩放特征
# transform应用此前的缩放参数，而不要重新计算
# last_n_scaled_features = scaler_features.transform(last_n_features)
features = daily_data_selected.drop(['Next_Open', 'P_news_pos', 'P_news_neg', 'P_op_pos', 'P_op_neg'], axis=1)


# 定义窗口长度
window_length = seq_length

# 创建包含所有窗口的数据集
batched_windows = []
for i in range(test_length - window_length + 1):
    window = last_n_scaled_features[i:i + window_length]
    batched_windows.append(window)
batched_windows = np.array(batched_windows)

# 将数据转换为Tensor
batched_windows_tensor = torch.tensor(batched_windows, dtype=torch.float32)

# 使用模型进行预测
model.eval()
with torch.no_grad():
    batch_predictions = model(batched_windows_tensor).numpy()

# 反缩放预测值
batch_predictions = scaler_target.inverse_transform(batch_predictions).flatten()

# 绘制实际股价和批量预测股价的对比图
plt.figure(figsize=(10, 6))
plt.plot(last_n_dates[window_length - 1:], last_n_target[window_length - 1:], label='Actual Prices', color='blue')
plt.plot(last_n_dates[window_length - 1:], batch_predictions, label='Batch Predicted Prices', color='red')
plt.title('Batch Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
# 滑动窗口预测
true_predictions = []
for i in range(test_length):
    if i < window_length:
        continue
    window = last_n_scaled_features[i-window_length:i]
    window_tensor = torch.tensor(window, dtype=torch.float32).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        prediction = model(window_tensor).numpy()
    true_prediction = scaler_target.inverse_transform(prediction).flatten()[0]
    true_predictions.append(true_prediction)
    
# 绘制第一张图 
plt.figure(figsize=(10, 6))
# TODO
plt.plot(last_n_dates, last_n_target, label='Actual Prices', color='blue')
plt.plot(last_n_dates[window_length:], true_predictions, label='Predicted Prices', color='red')
plt.title('Sliding Window Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
# 递归预测
recursive_predictions = last_n_scaled_features[-window_length:].tolist()  # 初始窗口
for i in range(test_length // 2):
    window_tensor = torch.tensor(recursive_predictions[-window_length:], dtype=torch.float32).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        prediction = model(window_tensor).numpy()
    recursive_predictions.append(prediction[0])

# 反缩放递归预测
recursive_predictions = scaler_target.inverse_transform(np.array(recursive_predictions).reshape(-1, 1)).flatten()

# 绘制第二张图
plt.figure(figsize=(10, 6))
plt.plot(last_n_dates, last_n_target, label='Actual Prices', color='blue')
plt.plot(last_n_dates, recursive_predictions[:test_length], label='Recursive Predicted Prices', color='red')
plt.title('Recursive Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


In [ ]:
# 假设您的模型已经被训练并准备好进行预测
# 我们将使用最后一个时间窗口的数据来预测接下来的n个时间点
# 为此，我们需要从数据集的末尾向前回溯一个窗口长度的数据点

# 定义窗口长度
window_length = 15  # 与您模型训练时使用的序列长度一致

# 预测未来n个时间点的函数
def predict_next_n(model, last_window_data, num_predictions=n):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    current_window = last_window_data
    
    with torch.no_grad():
        for _ in range(num_predictions):
            # 使用当前窗口数据进行预测
            current_window_tensor = torch.tensor(current_window, dtype=torch.float32).unsqueeze(0)  # 增加batch维度
            prediction = model(current_window_tensor)
            prediction = prediction.numpy().flatten()[0]  # 将预测结果转换为numpy并提取数值
            predictions.append(prediction)
            
            # 更新窗口数据：移除最早的点，加入最新预测
            current_window = np.roll(current_window, -1)
            current_window[-1] = prediction  # 这里假设您的目标是最后一个特征

    return predictions

# 获取最后一个时间窗口的数据
last_window_data = last_n_scaled_features[-window_length:]

# 预测下一个n个时间点
next_n_predictions_scaled = predict_next_n(model, last_window_data)

# 反缩放预测值
next_n_predictions = scaler_target.inverse_transform(np.array(next_n_predictions_scaled).reshape(-1, 1))

# 现在您有了未来n个时间点的预测，可以与实际数据进行比较
